In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Sentiment.csv')

In [3]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
tweet_df = df[['text','sentiment']]

In [5]:
tweet_df.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [6]:
tweet_df = tweet_df[tweet_df['sentiment'] != 'Neutral']

In [7]:
tweet_df.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [8]:
sentiment_label = tweet_df.sentiment.factorize()

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tweet = tweet_df.text.values
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen = 200)

In [11]:
print(tweet[0])
print(encoded_docs[0])

RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
[3, 365, 135, 2459, 1, 716, 2, 45, 65, 258, 42, 215, 7, 4368, 177, 1794, 18, 1339, 1430, 761, 13, 10, 11]


In [12]:
padded_sequence[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_length,input_length = 200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50,dropout = 0.5, recurrent_dropout = 0.5))
model.add(Dropout(0.2))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 32)           506240    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                16600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 522,891
Trainable params: 522,891
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
history = model.fit(padded_sequence, sentiment_label[0], validation_split = 0.2, epochs = 5, batch_size = 32)

Train on 8583 samples, validate on 2146 samples
Epoch 1/5
8583/8583 [==============================] - 36s 4ms/sample - loss: 0.5035 - acc: 0.7934 - val_loss: 0.4533 - val_acc: 0.8043
Epoch 2/5
8583/8583 [==============================] - 35s 4ms/sample - loss: 0.3749 - acc: 0.8404 - val_loss: 0.3532 - val_acc: 0.8234
Epoch 3/5
8583/8583 [==============================] - 35s 4ms/sample - loss: 0.3101 - acc: 0.8717 - val_loss: 0.3294 - val_acc: 0.8551
Epoch 4/5
8583/8583 [==============================] - 35s 4ms/sample - loss: 0.2762 - acc: 0.8875 - val_loss: 0.3350 - val_acc: 0.8551
Epoch 5/5
8583/8583 [==============================] - 35s 4ms/sample - loss: 0.2544 - acc: 0.8967 - val_loss: 0.3593 - val_acc: 0.8602


In [28]:
test_word = "Donald Trump is a bad person"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Negative'

In [35]:
pip install apriori_python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install apyori

  Using cached https://files.pythonhosted.org/packages/5e/62/5ffde5c473ea4b033490617ec5caa80d59804875ad3c3c57c0976533a21a/apyori-1.1.2.tar.gz
  Stored in directory: C:\Users\Suborno Deb\AppData\Local\pip\Cache\wheels\5d\92\bb\474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [1]:
from apyori import apriori

In [2]:
pip install bnltk

  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bnltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bnlp_toolkit

  Stored in directory: C:\Users\Suborno Deb\AppData\Local\pip\Cache\wheels\eb\83\5c\ead33ff91d363db5c2527b563746ba23887669c0221bd2484f
Successfully built smart-open
  Found existing installation: Cython 0.29.6
    Uninstalling Cython-0.29.6:
      Successfully uninstalled Cython-0.29.6
Note: you may need to restart the kernel to use updated packages.
